
# Домашнее задание № 11. Машинный перевод


## Задание 1 (6 баллов + 2 доп балла).
Нужно обучить трансформер на этом же или на другом корпусе (можно взять другую языковую пару с того же сайте) и оценивать его на всей тестовой выборке (а не на 10 примерах как сделал я). 

Чтобы получить 2 доп балла вам нужно будет придумать как оптимизировать функцию translate. Подсказка: модель может предсказывать батчами.

In [37]:
!pip install tokenizers matplotlib sklearn

In [75]:
import tensorflow as tf
from tokenizers import BertWordPieceTokenizer

from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.pre_tokenizers import Whitespace
from tokenizers import normalizers
from tokenizers.normalizers import Lowercase
from tokenizers.trainers import WordPieceTrainer
from tokenizers import decoders

import os
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from string import punctuation
from collections import Counter
from IPython.display import Image
from IPython.core.display import HTML 
import matplotlib.pyplot as plt
%matplotlib inline

In [76]:
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.ru
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.en
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.ru
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.en

--2022-05-22 18:25:45--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.ru
Resolving data.statmt.org (data.statmt.org)... 129.215.197.184
Connecting to data.statmt.org (data.statmt.org)|129.215.197.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 121340806 (116M)
Saving to: ‘opus.en-ru-train.ru.2’

opus.en-ru-train.ru 100%[===================>] 115.72M  95.8MB/s    in 1.2s    

2022-05-22 18:25:46 (95.8 MB/s) - ‘opus.en-ru-train.ru.2’ saved [121340806/121340806]

--2022-05-22 18:25:46--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.en
Resolving data.statmt.org (data.statmt.org)... 129.215.197.184
Connecting to data.statmt.org (data.statmt.org)|129.215.197.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67760131 (65M)
Saving to: ‘opus.en-ru-train.en.2’

opus.en-ru-train.en 100%[===================>]  64.62M  89.1MB/s    in 0.7s    

2022-05-22 18:25:47 (89.1 MB/s)

In [77]:
en_sents = open('opus.en-ru-train.en').read().lower().splitlines()
ru_sents = open('opus.en-ru-train.ru').read().lower().splitlines()

In [78]:
tokenizer_en = Tokenizer(WordPiece(), )
tokenizer_en.normalizer = normalizers.Sequence([Lowercase()])
tokenizer_en.pre_tokenizer = Whitespace()

trainer_en = WordPieceTrainer(vocab_size=30000,
                              special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]"])
tokenizer_en.train(files=["opus.en-ru-train.en"],
                   trainer=trainer_en )

tokenizer_ru = Tokenizer(WordPiece(), )
tokenizer_ru.normalizer = normalizers.Sequence([Lowercase()])
tokenizer_ru.pre_tokenizer = Whitespace()

trainer_ru = WordPieceTrainer(vocab_size=30000,
                              special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]"])
tokenizer_ru.train(files=["opus.en-ru-train.ru"],
                   trainer=trainer_ru )

In [79]:
tokenizer_en.decoder = decoders.WordPiece()
tokenizer_ru.decoder = decoders.WordPiece()

In [80]:
def encode(text, tokenizer, target=False):
    return [tokenizer.token_to_id('[CLS]')] + tokenizer.encode(text).ids + [tokenizer.token_to_id('[SEP]')]

In [81]:
X_en = [encode(t, tokenizer_en) for t in en_sents]
X_ru = [encode(t, tokenizer_ru, target=True) for t in ru_sents]

In [82]:
max_len_en, max_len_ru = 20, 22

In [83]:
PAD_IDX = tokenizer_en.token_to_id('[PAD]')

In [84]:
X_en = tf.keras.preprocessing.sequence.pad_sequences(X_en,
                                                     maxlen=max_len_en,
                                                     padding='post',
                                                     value=tokenizer_en.token_to_id('[PAD]'))

X_ru_out = tf.keras.preprocessing.sequence.pad_sequences([x[1:] for x in X_ru],
                                                         maxlen=max_len_ru-1, padding='post', 
                                                         value=tokenizer_en.token_to_id('[PAD]'))

X_ru_dec = tf.keras.preprocessing.sequence.pad_sequences([x[:-1] for x in X_ru],
                                                         maxlen=max_len_ru-1,
                                                         padding='post', value=tokenizer_ru.token_to_id('[PAD]'))

In [85]:
X_en_train, X_en_valid, X_ru_dec_train, X_ru_dec_valid, X_ru_out_train, X_ru_out_valid = train_test_split(X_en, 
                                                                                                          X_ru_dec, 
                                                                                                          X_ru_out, 
                                                                                                          test_size=0.05)

In [86]:
def scaled_dot_product_attention(query, key, value, mask):
    """Calculate the attention weights. """
    matmul_qk = tf.matmul(query, key, transpose_b=True)

    # scale matmul_qk
    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth)

    # add the mask to zero out padding tokens
    if mask is not None:
        logits += (mask * -1e9)

    # softmax is normalized on the last axis (seq_len_k)
    attention_weights = tf.nn.softmax(logits, axis=-1)

    output = tf.matmul(attention_weights, value)

    return output

In [87]:
class MultiHeadAttention(tf.keras.layers.Layer):

    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)

        self.dense = tf.keras.layers.Dense(units=d_model)

    def split_heads(self, inputs, batch_size):
        inputs = tf.reshape(
            inputs, shape=(batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(inputs, perm=[0, 2, 1, 3])

    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs[
            'value'], inputs['mask']
        batch_size = tf.shape(query)[0]

        # linear layers
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        # split heads
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        # scaled dot-product attention
        scaled_attention = scaled_dot_product_attention(query, key, value, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        # concatenation of heads
        concat_attention = tf.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))

        # final linear layer
        outputs = self.dense(concat_attention)

        return outputs

In [88]:
def create_padding_mask(x):
    mask = tf.cast(tf.math.equal(x, PAD_IDX), tf.float32)
    # (batch_size, 1, 1, sequence length)
    return mask[:, tf.newaxis, tf.newaxis, :]

In [89]:
def create_look_ahead_mask(x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    padding_mask = create_padding_mask(x)
    return tf.maximum(look_ahead_mask, padding_mask)

In [90]:
class PositionalEncoding(tf.keras.layers.Layer):

    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return position * angles

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)
        sines = tf.math.sin(angle_rads[:, 0::2])
        cosines = tf.math.cos(angle_rads[:, 1::2])

        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)

    def call(self, inputs):

        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

In [91]:
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })
    attention = tf.keras.layers.Dropout(rate=dropout)(attention)
    attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

    return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [92]:
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            max_len,
            name="encoder"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(max_len, d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = encoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name="encoder_layer_{}".format(i),
        )([outputs, padding_mask])

    return tf.keras.Model(inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [93]:
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
    look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })
    attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

    attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1,
          'key': enc_outputs,
          'value': enc_outputs,
          'mask': padding_mask
      })
    attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
    attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

    return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [94]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            max_len,
            name='decoder'):
    inputs = tf.keras.Input(shape=(None,), name='inputs')
    enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
    look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(max_len, d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = decoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name='decoder_layer_{}'.format(i),
        )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

    return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [95]:
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                max_len,
                name="transformer"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

    enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)
    # mask the future tokens for decoder inputs at the 1st attention block
    look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask,
      output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)
    # mask the encoder outputs for the 2nd attention block
    dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

    enc_outputs = encoder(
      vocab_size=vocab_size[0],
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
      max_len=max_len[0],
    )(inputs=[inputs, enc_padding_mask])

    dec_outputs = decoder(
      vocab_size=vocab_size[1],
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
      max_len=max_len[1],
    )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

    outputs = tf.keras.layers.Dense(units=vocab_size[1], name="outputs")(dec_outputs)

    return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

In [96]:
L  = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none',)

def loss_function(y_true, y_pred):
    loss = L(y_true, y_pred)

    mask = tf.cast(tf.not_equal(y_true, PAD_IDX), tf.float32)
    loss = tf.multiply(loss, mask)

    return tf.reduce_mean(loss)

In [97]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [98]:
tf.keras.backend.clear_session()

# small model
NUM_LAYERS = 2
D_MODEL = 256
NUM_HEADS = 8
UNITS = 512
DROPOUT = 0.1


mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    model = transformer(
        vocab_size=(tokenizer_en.get_vocab_size(),tokenizer_ru.get_vocab_size()),
        num_layers=NUM_LAYERS,
        units=UNITS,
        d_model=D_MODEL,
        num_heads=NUM_HEADS,
        dropout=DROPOUT,
        max_len=[max_len_en, max_len_ru])

#     learning_rate = CustomSchedule(D_MODEL)

    optimizer = tf.keras.optimizers.Adam(
        0.001, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

    def accuracy(y_true, y_pred):
#         y_true = tf.reshape(y_true, shape=(-1, max_len_ru - 1))
        return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)


    model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])
    checkpoint = tf.keras.callbacks.ModelCheckpoint('model_ruen',
                                                monitor='val_loss',
                                                verbose=1,
                                            save_weights_only=True,
                                            save_best_only=True,
                                            mode='min',
                                            save_freq='epoch')

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [99]:
model.fit((X_en_train, X_ru_dec_train), X_ru_out_train, 
             validation_data=((X_en_valid, X_ru_dec_valid), X_ru_out_valid),
             batch_size=512,
             epochs=6,
             callbacks=[checkpoint]
             )

Epoch 1/6
1856/1856 [==============================] - ETA: 0s - loss: 2.4584 - accuracy: 0.2022
Epoch 1: val_loss improved from inf to 1.79796, saving model to model_ruen
1856/1856 [==============================] - 719s 383ms/step - loss: 2.4584 - accuracy: 0.2022 - val_loss: 1.7980 - val_accuracy: 0.2634
Epoch 2/6
1856/1856 [==============================] - ETA: 0s - loss: 1.6788 - accuracy: 0.2736
Epoch 2: val_loss improved from 1.79796 to 1.57780, saving model to model_ruen
1856/1856 [==============================] - 708s 381ms/step - loss: 1.6788 - accuracy: 0.2736 - val_loss: 1.5778 - val_accuracy: 0.2876
Epoch 3/6
1856/1856 [==============================] - ETA: 0s - loss: 1.4996 - accuracy: 0.2933
Epoch 3: val_loss improved from 1.57780 to 1.49602, saving model to model_ruen
1856/1856 [==============================] - 708s 382ms/step - loss: 1.4996 - accuracy: 0.2933 - val_loss: 1.4960 - val_accuracy: 0.2984
Epoch 4/6
1856/1856 [==============================] - ETA: 0s - 

In [100]:
model.save('drive/MyDrive/model')

INFO:tensorflow:Assets written to: drive/MyDrive/model/assets


INFO:tensorflow:Assets written to: drive/MyDrive/model/assets


In [29]:
model.load_weights('drive/MyDrive/model')

In [101]:
from itertools import islice
from tqdm import tqdm


def chunk(it, size):
    it = iter(it)
    return list(iter(lambda: tuple(islice(it, size)), ()))

In [102]:
def translate(texts, batch_size=128):
  res = []
  batches = chunk(texts, batch_size)
  sep = tokenizer_ru.token_to_id('[SEP]')
  
  for batch in tqdm(batches):
    res_sent = []

    input_ids = [encode(text.lower(), tokenizer_en) for text in batch]
    input_ids = tf.keras.preprocessing.sequence.pad_sequences(input_ids,
                                                              maxlen=max_len_en,
                                                              padding='post')
    output_ids = [[tokenizer_ru.token_to_id('[CLS]')] for _ in range(len(input_ids))]

    preds = model.predict((input_ids, tf.cast(output_ids, tf.int32)),
                          batch_size=batch_size).argmax(2)[:, -1]
    
    for i in range(max_len_ru - 1):
      output_ids = np.column_stack([output_ids, preds])
      if i == max_len_ru - 2:
        for output in output_ids:
          if len(np.where(output==sep)[0]) == 0:
            last_ind = -1
          else:
            last_ind = np.where(output==sep)[0][0]
          res_sent.append(tokenizer_ru.decode(output[1:last_ind]))
      else:
        preds = model.predict((input_ids, tf.cast(output_ids, tf.int32)),
                            batch_size=32).argmax(2)[:, -1]

    res.extend(res_sent)

  return res

In [103]:
en_sents_test = open('opus.en-ru-test.en').read().lower().splitlines()
ru_sents_test = open('opus.en-ru-test.ru').read().lower().splitlines()

In [104]:
translations = translate(en_sents_test)

100%|██████████| 16/16 [01:49<00:00,  6.83s/it]


In [105]:
import nltk

bleus = []

for i, sentence in enumerate(translations):
  reference = tokenizer_ru.encode(sentence).tokens
  hypothesis = tokenizer_ru.encode(ru_sents_test[i]).tokens

bleus.append(nltk.translate.bleu_score.sentence_bleu([reference], hypothesis,  ))

In [107]:
(sum(bleus)/len(bleus))*100

39.806687382400554

## Задание 2 (2 балла).
Прочитайте главу про машинный перевод у Журафски и Маннига - https://web.stanford.edu/~jurafsky/slp3/10.pdf 
Ответьте своими словами в чем заключается техника back translation? Для чего она применяется и что позволяет получить? Опишите по шагам как его применить к паре en-ru на данных из семинара. 

Back translation - метод, при котором создаются дополнительные синтетические данные, он достаточно легок в применении, т.к. не требует модификации алгоритмов обучения MT. Такой метод позволяет получить дополнительные данные тогда, когда имеется большой корпус одного языка, а параллельный корпус при этом маленький. Алгоритм back translation  такой: исходные моноязычные данные переводятся на целевой язык, целевые моноязычные данные переводятся обратно, а затем эти синтетические данные добавляются к настоящим двуязычным данным. 
Для пары en-ru:
- На нашем параллельном корпусе обучаем переводить модель с русского на английский 
- Генерируем предложения на английском языке по предложениям на русском языке с помощью различных параметров (например, Monte Carlo search)
- Полученнные сгенерированные предложения добавляем в наш параллельный корпус 
- На полученном датасете обучаем МТ-модель на перевод с английского на русский язык